In [1]:
import ee
import geemap
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm 
import time
from datetime import datetime

In [2]:
geemap.set_proxy(port=7890) # set your proxy port
ee.Authenticate()
ee.Initialize(project='socd-liuziyan') # change your GEE project name

modeling socd by RF

In [3]:
soc_data = ee.FeatureCollection('projects/socd-liuziyan/assets/1_final_socd_0_100cm')

embedding_dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
embedding_2024 = embedding_dataset.filterDate('2024-01-01', '2025-01-01').mosaic()

In [4]:
# Split the SOC data into training (80%) and validation (20%) sets
soc_data = soc_data.randomColumn(columnName='random', seed=42)
training_fc = soc_data.filter(ee.Filter.lt('random', 0.8))
validation_fc = soc_data.filter(ee.Filter.gte('random', 0.8))

# Sample the embedding image at the locations of the SOC data
training = embedding_2024.sampleRegions(
    collection=training_fc, properties=['socd_100cm'], scale=10, tileScale=16
)
test = embedding_2024.sampleRegions(
    collection=validation_fc, properties=['socd_100cm'], scale=10, tileScale=16
)

# Tune the number of trees in Random Forest
band_names = embedding_2024.bandNames()
numTreesList = ee.List.sequence(0, 200, 20)

In [5]:
optimal_model = ee.Classifier.smileRandomForest(
    numberOfTrees=160,
    variablesPerSplit=8,
    minLeafPopulation=5,
    bagFraction=0.8,
    seed=42
).setOutputMode('REGRESSION').train(
    features=training,
    classProperty='socd_100cm',
    inputProperties=band_names
)

In [6]:
# mountain_mask = ee.Image('projects/socd-liuziyan/assets/global_mountain_classification_UNEP')
# kernel = ee.Kernel.circle(radius=2500, units='meters', normalize=True)
# dilated = mountain_mask.focal_max(kernel=kernel, iterations=1)
# closed_mask = dilated.focal_min(kernel=kernel, iterations=1)

# embedding_2024_masked = embedding_2024.updateMask(closed_mask)
# orgc_prediction = embedding_2024_masked.classify(optimal_model)
# orgc_prediction = orgc_prediction.rename('socd_100cm_prediction')

In [7]:
orgc_prediction = embedding_2024.classify(optimal_model)
orgc_prediction = orgc_prediction.rename('socd_100cm_prediction')

In [8]:
Map = geemap.Map(center=[20,0], zoom=2)
prediction_vis = {
    'min': 0,
    'max': 50,
    'palette': [    
        '#543005', '#bc8120', '#dfc27d', '#f6e8c3',
        '#f5f5f5', '#c7eae5', '#80cdc1', '#35978f',
        '#01665e', '#003c30'
    ]
}
Map.addLayer(orgc_prediction, prediction_vis, 'SOCD Prediction')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [9]:
## process SoilGrids soc data for downloading
ocd = ee.Image('projects/soilgrids-isric/ocd_mean').toFloat()

ocd_stock_0_100cm = (
    ocd.select('ocd_0-5cm_mean').multiply(0.05)
      .add(ocd.select('ocd_5-15cm_mean').multiply(0.10))
      .add(ocd.select('ocd_15-30cm_mean').multiply(0.15))
      .add(ocd.select('ocd_30-60cm_mean').multiply(0.30))
      .add(ocd.select('ocd_60-100cm_mean').multiply(0.40))
).rename('ocd_stock_0_100cm').multiply(0.1)

Map.addLayer(ocd_stock_0_100cm, prediction_vis, 'SoilGrids SOCD 0-100cm')

downloading for dayekou in China

In [10]:
# dayekou_coords = [
#     [99.73661832827517, 38.399959125038414],
#     [100.11290006655642, 38.399959125038414],
#     [100.11290006655642, 38.67119851515649],
#     [99.73661832827517, 38.67119851515649],
#     [99.73661832827517, 38.399959125038414],
# ]
# dayekou_geom = ee.Geometry.Polygon([dayekou_coords], None, False).bounds()

# fishnet = geemap.fishnet(dayekou_geom, rows=8, cols=8)

# out_dir = os.path.join(os.getcwd(), 'downloads', 'socd_10m_dayekou_tiles')

# orgc_dayekou = orgc_prediction.clip(dayekou_geom)
# geemap.download_ee_image_tiles(
#     image=orgc_dayekou,
#     features=fishnet,
#     out_dir=out_dir,
#     prefix="socd_10m_dayekou_",
#     crs="EPSG:4326",
#     scale=10,
#     num_threads=10,
#     max_tile_size=16,
#     max_tile_dim=512
# )

In [11]:
# min_lon, min_lat = 100.1635914837812464, 38.4275024033660131
# max_lon, max_lat = 100.4239232531190709, 38.6832527647548261
# dayekou_geom = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat], None, False)

# out_dir = os.path.join(os.getcwd(), 'downloads', 'soilgrids_socd_stock_dayekou')
# os.makedirs(out_dir, exist_ok=True)
# out_file = os.path.join(out_dir, 'socd_stock_0_100cm_dayekou.tif')

# geemap.download_ee_image(
#     image=ocd_stock_0_100cm.clip(dayekou_geom),
#     filename=out_file,
#     region=dayekou_geom,
#     scale=250,         
#     crs='EPSG:4326'
# )

downloading Tianshan Mountains in China

In [12]:
tianshan_coords = [
        [85.6630502529307, 43.79791613113733],
        [85.6630502529307, 43.509770583735005],
        [86.09975679589945, 43.509770583735005],
        [86.09975679589945, 43.79791613113733],
]
tianshan_geom = ee.Geometry.Polygon([tianshan_coords], None, False).bounds()

fishnet = geemap.fishnet(tianshan_geom, rows=4, cols=4)

out_dir = os.path.join(os.getcwd(), 'downloads', 'socd_10m_tianshan_tiles')
# orgc_tianshan = orgc_prediction.clip(tianshan_geom)
geemap.download_ee_image_tiles(
    image=orgc_prediction,
    features=fishnet,
    out_dir=out_dir,
    crs="EPSG:4326",
    prefix="socd_10m_tianshan_",
    scale=10,
    num_threads=10,
    max_tile_size=16,
    max_tile_dim=1024
)

socd_10m_tianshan_01.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


socd_10m_tianshan_02.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_tianshan_03.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_tianshan_04.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

...

socd_10m_tianshan_22.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_tianshan_23.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_tianshan_24.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_tianshan_25.tif: |          | 0.00/3.91M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 25 tiles in 1265.0548841953278 seconds.


In [15]:
# min_lon, min_lat = 100.1635914837812464, 38.4275024033660131
# max_lon, max_lat = 100.4239232531190709, 38.6832527647548261
# dayekou_geom = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat], None, False)

out_dir = os.path.join(os.getcwd(), 'downloads', 'soilgrids_socd_stock_tianshan')
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'socd_stock_0_100cm_tianshan.tif')

geemap.download_ee_image(
    image=ocd_stock_0_100cm,
    filename=out_file,
    region=tianshan_geom.bounds(),
    scale=250,
    crs='EPSG:4326'
)

socd_stock_0_100cm_tianshan.tif: |          | 0.00/204k (raw) [  0.0%] in 00:00 (eta:     ?)

download for western us

In [18]:
westUS_coords = [
        [-115.63772620349806, 45.19657840325652],
        [-115.63772620349806, 44.57386323138948],
        [-114.52810706287306, 44.57386323138948],
        [-114.52810706287306, 45.19657840325652],
]

westUS_geom = ee.Geometry.Polygon([westUS_coords], None, False).bounds()

fishnet = geemap.fishnet(westUS_geom, rows=3, cols=3)

out_dir = os.path.join(os.getcwd(), 'downloads', 'socd_10m_westUS_tiles')
# orgc_westUS = orgc_prediction.clip(westUS_geom)
geemap.download_ee_image_tiles(
    image=orgc_prediction,
    features=fishnet,
    out_dir=out_dir,
    crs="EPSG:4326",
    prefix="socd_10m_westUS_",
    scale=10,
    num_threads=10,
    max_tile_size=16,
    max_tile_dim=1024
)

socd_10m_westUS_01.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_02.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_03.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_04.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_05.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

...

socd_10m_westUS_13.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_14.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_15.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_westUS_16.tif: |          | 0.00/38.1M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 16 tiles in 1948.0103375911713 seconds.


In [19]:
out_dir = os.path.join(os.getcwd(), 'downloads', 'soilgrids_socd_stock_westUS')
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'socd_stock_0_100cm_westUS.tif')

geemap.download_ee_image(
    image=ocd_stock_0_100cm,
    filename=out_file,
    region=westUS_geom.bounds(),
    scale=250,
    crs='EPSG:4326'
)

socd_stock_0_100cm_westUS.tif: |          | 0.00/1.10M (raw) [  0.0%] in 00:00 (eta:     ?)

sample 3 in Russa

In [22]:
region4_coords = [
        [98.5594275632723, 48.85888041258772],
        [98.5594275632723, 48.33209929845491],
        [99.48639899881917, 48.33209929845491],
        [99.48639899881917, 48.85888041258772]
]

region4_geom = ee.Geometry.Polygon([region4_coords], None, False).bounds()

fishnet = geemap.fishnet(region4_geom, rows=3, cols=3)

out_dir = os.path.join(os.getcwd(), 'downloads', 'socd_10m_region4_tiles')
# orgc_westUS = orgc_prediction.clip(westUS_geom)
geemap.download_ee_image_tiles(
    image=orgc_prediction,
    features=fishnet,
    out_dir=out_dir,
    crs="EPSG:4326",
    prefix="socd_10m_region4_",
    scale=10,
    num_threads=10,
    max_tile_size=16,
    max_tile_dim=1024
)

socd_10m_region4_01.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_02.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_03.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_04.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_05.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

...

socd_10m_region4_13.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_14.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_15.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

socd_10m_region4_16.tif: |          | 0.00/26.9M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 16 tiles in 990.2184679508209 seconds.


In [23]:
out_dir = os.path.join(os.getcwd(), 'downloads', 'soilgrids_socd_stock_region4')
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'socd_stock_0_100cm_region4.tif')

geemap.download_ee_image(
    image=ocd_stock_0_100cm,
    filename=out_file,
    region=region4_geom.bounds(),
    scale=250,
    crs='EPSG:4326'
)

socd_stock_0_100cm_region4.tif: |          | 0.00/778k (raw) [  0.0%] in 00:00 (eta:     ?)